# Obtain edges from a Pajek-like file

In [50]:
import time
import pandas as pd
import numpy as np

In [51]:
s=time.time()
imax=100#000
for i in range(imax):
    for j in range(imax):
        a=1+1

print(time.time()-s)        

0.001165628433227539


In [52]:
import pandas as pd

##  Obtain full nodes file

In [461]:
df=pd.read_json('nodes_small.json')

In [462]:
df[:1]

,abstract,authors,citations,co-authors,creation_date,free_keywords,list_rcid,recid,references,standardized_keywords,title
0,The exploitation of unrenormalized canonical f...,"[Brandt, Richard A.]",[18551],"[Ng, Wing-Chiu, Young, Kenneth]",1976-05,[],"[3560, 3560, 3560, 3560, 3560, 3560, 3560, 356...",3560,"[87072, 1283, 86276, 1577, 95084, 54444, 99663...","[GAUGE FIELD THEORY: NONABELIAN, GAUGE FIELD T...",Canonical Methods in Nonabelian Gauge Theories


## 

In [463]:
pj=pd.DataFrame( { 'recid': np.concatenate (  tuple( df.list_rcid.values ) ),
                   'references': np.concatenate (  tuple( df.references.values ) )} )
pj=pj.sort_values(['references','recid'])

In [464]:
pj.shape

(23960, 2)

### Remove entries with a single citation

In [465]:
pj['diff_refs_1']=np.concatenate (  (  pj.references.values[1:]-pj.references.values[:-1],
                            [   pj.references.values[-1]-pj.references.values[-2]   ] ) )
pj['diff_refs_2']=np.concatenate (  (  [   pj.references.values[-1]-pj.references.values[-2]   ],
                                         (pj.references.values[:-1]-pj.references.values[1:])
                                   ) )
pj['diff_refs']=pj.diff_refs_1*pj.diff_refs_2

In [466]:
pj=pj[pj.diff_refs==0].drop(['diff_refs_1','diff_refs_2','diff_refs'],axis='columns').reset_index(drop=True)

In [467]:
#pj=pjj[0:7].reset_index(drop=True)
pj.shape

(4915, 2)

##  Main code to find the edges

In [468]:
id_col='recid'; refs_col='references'
pj=pj.sort_values([refs_col,id_col]).reset_index(drop=True)
ed={}
t=()
new_ref=0
i=0
iprint=1000
while i<=pj.index[-1]:
    if not i%iprint:
        print(i,end='\r')
    e1=pj.loc[i,id_col]
    #check next entry and beyond:
    for j in range(i+1,pj.index[-1]+1):
        #print('i,j',i,j) #,end='\r')
        e2=pj.loc[j,id_col]
        #DEBUG
        #if (e1,e2)==(190836,1644265):
        #    print('i,j,check',i,j)
        edges=False
        if pj.loc[i,refs_col]==pj.loc[j,refs_col]:
            edges=True
        #else:
            #print('newi',i,j)
            
        if edges:
            if e1<e2:
                t=(e1,e2)
                if not ed.get(t):
                    ed[t]=1
                    #DEBUG
                    #ed[t]=str( (i,j) )
                else: #increase the number of common refs_col
                    ed[t]=ed[t]+1
                    #DEBUG: 
                    #ed[t]=ed[t]+str(  (i,j)  )
        else:
            # Go to next i when nor additional edges are found
            break
    #search after j. 
    i=i+1 # Be sure to increase i to avoid infinite loop
    #if i>=25:
    #    break
    continue

In [469]:
le=[{t:ed[t]} for t in ed.keys() if ed[t]>4 ]
len(le)

124

In [459]:
edf=pd.DataFrame({'tmp':le})
edf['edges'] =edf.tmp.map(lambda x: list(x.keys())[0]  )
edf['len_ed']=edf.tmp.map(lambda x: list(x.values())[0]  )
edf=edf.drop('tmp',axis='columns')
edf[:3]

,edges,len_ed
0,"(480018, 562787)",9
1,"(719594, 1434935)",6
2,"(1296453, 1434935)",5


In [425]:
pjj[0:7]

,recid,references
0,190836,2724
1,823215,2724
2,1081640,2724
3,1260873,2724
4,1644265,2724
5,190836,130473
6,823215,130473


In [391]:
pjj[np.logical_or( pjj.recid==823215,pjj.recid==1081640 )]

,recid,references
1,823215,2724
2,1081640,2724
6,823215,130473
7,1081640,130473
11,1081640,140094
14,1081640,142415


In [393]:
ed.get(  (823215,1081640) )

In [398]:
ed

{(190836, 823215): 2,
 (190836, 1081640): 1,
 (190836, 1260873): 1,
 (190836, 1644265): 1}

In [399]:
pjj[0:7]

,recid,references
0,190836,2724
1,823215,2724
2,1081640,2724
3,1260873,2724
4,1644265,2724
5,190836,130473
6,823215,130473


In [381]:
pjj[15:]#.reset_index(drop=True)
#pjj[np.logical_or( pjj.recid==190836,pjj.recid==1644265 ) ]

,recid,references
15,1644265,142415
16,24826,159286
17,190836,159286
18,1644265,159286
19,190836,165176
20,884964,165176
21,1229048,165176
22,1246799,165176
23,1355396,165176
24,1644265,165176


In [368]:
(190836,1644265)==(190836,1644265)

True

In [375]:
ed.get(  (190836,1644265) )

1

In [311]:
pjj

,recid,references
0,190836,2724
1,823215,2724
2,1081640,2724
3,1260873,2724
4,1644265,2724
5,190836,130473
6,823215,130473
7,1081640,130473
8,1260873,130473
9,1644265,130473


check 

In [240]:
edfin=pd.read_json('edges.json')

In [242]:
edfin['list_edges']=edfin[['source','target']].values.tolist()
edfin['list_edges']=edfin['list_edges'].map(str)

In [246]:
edfin['len_edfin']=edfin.intersect.map(len)

In [258]:
edfin[:1]

,intersect,source,target,list_edges,len_edfin
0,"[1327, 6860, 100686, 123323, 144015, 190671, 2...",24966,317793,"[24966, 317793]",7


In [261]:
sl=edfin[['list_edges','intersect']].merge(edf[['list_edges','len_ed']],on='list_edges',how='left')

In [285]:
pj[pj.recid==190836]

,recid,references
0,190836,2724
5,190836,130473
10,190836,140094
13,190836,142415
17,190836,159286
19,190836,165176


In [288]:
pj[pj.recid==1644265]

,recid,references
4,1644265,2724
9,1644265,130473
12,1644265,140094
15,1644265,142415
18,1644265,159286
24,1644265,165176


In [301]:
pjj=pd.DataFrame()
for r in pj[pj.recid==190836].references:
    pjj=pjj.append(  pj[pj.references==r]    )

In [291]:
pj[pj.references==2724]

,recid,references
0,190836,2724
1,823215,2724
2,1081640,2724
3,1260873,2724
4,1644265,2724


In [295]:
pj[pj.references==130473]

,recid,references
5,190836,130473
6,823215,130473
7,1081640,130473
8,1260873,130473
9,1644265,130473


In [297]:
pj[pj.references==140094]

,recid,references
10,190836,140094
11,1081640,140094
12,1644265,140094


In [298]:
pj[pj.references==140094] 

,recid,references
10,190836,140094
11,1081640,140094
12,1644265,140094


In [299]:
pj[pj.references==159286] 

,recid,references
16,24826,159286
17,190836,159286
18,1644265,159286


In [300]:
pj[pj.references==165176] 

,recid,references
19,190836,165176
20,884964,165176
21,1229048,165176
22,1246799,165176
23,1355396,165176
24,1644265,165176


In [302]:
pjj

,recid,references
0,190836,2724
1,823215,2724
2,1081640,2724
3,1260873,2724
4,1644265,2724
5,190836,130473
6,823215,130473
7,1081640,130473
8,1260873,130473
9,1644265,130473


IMPORTANT COMMITS: 
* 44fa444ba3cc1f9a052bf80051f44a0d11c5838a
  * Here the counting of the slow and fast to grouping mathc
